In [1]:
import pandas as pd
import numpy as np
from os import listdir
from tqdm.auto import tqdm
tqdm.pandas()

from random import choice

In [2]:
from agent_code.my_agent.features.movement_graph import MovementGraph
from agent_code.my_agent.features.feature import FeatureCollector
from agent_code.my_agent.features.actions import Actions

In [3]:
batch_files = [f for f in listdir("batches") if f.startswith("batch")]
batches = len(batch_files) * [None]

for batch_file in tqdm(batch_files):
    batch_number = int(batch_file.split("_")[1].split(".")[0]) 

    batch = pd.read_pickle(f"batches/{batch_file}")
    batch["batch_number"] = batch_number
    # already sort by batch number
    batches[batch_number - 1] = batch
    
assert not any(b is None for b in batches)

last_round = 0

for i, batch in tqdm(enumerate(batches)):
    batch = batch.rename({"round": "rounds"}, axis=1)
    batch.rounds += last_round
    last_round = batch.rounds.max()
    
    batches[i] = batch

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [4]:
data = pd.concat(batches).reset_index(drop=True)

In [5]:
# zero indexed, does not exists
data = data.drop(labels=["agent_4_state", "agent_4_action"], axis=1)

In [6]:
print(data.isna().sum())

data = data.fillna("NONE")

print(data.isna().sum())

rounds               0
step                 0
field                0
bombs                0
coins                0
agent_1_state        0
agent_1_action    5570
agent_2_state        0
agent_2_action    3698
agent_3_state        0
agent_3_action    1612
agent_0_action    6016
agent_0_state        0
explosion_map        0
batch_number         0
dtype: int64
rounds            0
step              0
field             0
bombs             0
coins             0
agent_1_state     0
agent_1_action    0
agent_2_state     0
agent_2_action    0
agent_3_state     0
agent_3_action    0
agent_0_action    0
agent_0_state     0
explosion_map     0
batch_number      0
dtype: int64


In [7]:
# convert action strings to actrion enum values
for agent in range(4):
    col_name = f"agent_{agent}_action"
    data[col_name] = data[col_name].apply(lambda x: Actions[x])

In [8]:
for shift_by in tqdm(range(1, 5)):
    shifted = data.groupby(["rounds"])\
        .shift(shift_by)\
        .fillna(Actions.NONE)
    you
    for agent in range(4):
        data[f"agent_{agent}_move_{shift_by}_step_ago"] = shifted[f"agent_{agent}_action"]

Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
data

,rounds,step,field,bombs,coins,agent_1_state,agent_1_action,agent_2_state,agent_2_action,agent_3_state,...,agent_2_move_2_step_ago,agent_3_move_2_step_ago,agent_0_move_3_step_ago,agent_1_move_3_step_ago,agent_2_move_3_step_ago,agent_3_move_3_step_ago,agent_0_move_4_step_ago,agent_1_move_4_step_ago,agent_2_move_4_step_ago,agent_3_move_4_step_ago
0,1,1,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...",[],[],"(rule_based_agent_1, 0, True, (15, 1))",Actions.DOWN,"(rule_based_agent_2, 0, True, (1, 1))",Actions.RIGHT,"(rule_based_agent_3, 0, True, (1, 15))",...,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE
1,1,2,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...",[],[],"(rule_based_agent_1, 0, True, (15, 2))",Actions.DOWN,"(rule_based_agent_2, 0, True, (2, 1))",Actions.BOMB,"(rule_based_agent_3, 0, True, (1, 14))",...,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE
2,1,3,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[((1, 14), 3), ((2, 1), 3), ((15, 14), 3)]",[],"(rule_based_agent_1, 0, True, (15, 3))",Actions.BOMB,"(rule_based_agent_2, 0, False, (2, 1))",Actions.LEFT,"(rule_based_agent_3, 0, False, (1, 14))",...,Actions.RIGHT,Actions.UP,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE
3,1,4,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[((1, 14), 2), ((2, 1), 2), ((15, 14), 2), ((1...",[],"(rule_based_agent_1, 0, False, (15, 3))",Actions.UP,"(rule_based_agent_2, 0, False, (1, 1))",Actions.DOWN,"(rule_based_agent_3, 0, False, (1, 15))",...,Actions.BOMB,Actions.BOMB,Actions.UP,Actions.DOWN,Actions.RIGHT,Actions.UP,Actions.NONE,Actions.NONE,Actions.NONE,Actions.NONE
4,1,5,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[((1, 14), 1), ((2, 1), 1), ((15, 14), 1), ((1...",[],"(rule_based_agent_1, 0, False, (15, 2))",Actions.UP,"(rule_based_agent_2, 0, False, (1, 2))",Actions.UP,"(rule_based_agent_3, 0, False, (2, 15))",...,Actions.LEFT,Actions.DOWN,Actions.BOMB,Actions.DOWN,Actions.BOMB,Actions.BOMB,Actions.UP,Actions.DOWN,Actions.RIGHT,Actions.UP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980452,2800,23,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...",[],"[(1, 4)]","(random_agent_1, 0, True, (14, 15))",Actions.WAIT,"(random_agent_2, 0, True, (1, 15))",Actions.WAIT,"(random_agent_3, 0, True, (14, 1))",...,Actions.WAIT,Actions.LEFT,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.LEFT,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.RIGHT
980453,2800,24,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[((14, 1), 3)]","[(1, 4)]","(random_agent_1, 0, True, (14, 15))",Actions.WAIT,"(random_agent_2, 0, True, (1, 15))",Actions.WAIT,"(random_agent_3, 0, False, (14, 1))",...,Actions.WAIT,Actions.DOWN,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.LEFT,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.LEFT
980454,2800,25,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[((14, 1), 2)]","[(1, 4)]","(random_agent_1, 0, True, (14, 15))",Actions.WAIT,"(random_agent_2, 0, True, (1, 15))",Actions.WAIT,"(random_agent_3, 0, False, (14, 1))",...,Actions.WAIT,Actions.BOMB,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.DOWN,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.LEFT
980455,2800,26,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[((14, 1), 1)]","[(1, 4)]","(random_agent_1, 0, True, (14, 15))",Actions.WAIT,"(random_agent_2, 0, True, (1, 15))",Actions.WAIT,"(random_agent_3, 0, False, (14, 1))",...,Actions.WAIT,Actions.UP,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.BOMB,Actions.WAIT,Actions.WAIT,Actions.WAIT,Actions.DOWN


In [35]:
data.apply(lambda x: print(type(x), x))

<class 'pandas.core.series.Series'> 0            1
1            1
2            1
3            1
4            1
          ... 
980452    2800
980453    2800
980454    2800
980455    2800
980456    2800
Name: rounds, Length: 980457, dtype: int64
<class 'pandas.core.series.Series'> 0          1
1          2
2          3
3          4
4          5
          ..
980452    23
980453    24
980454    25
980455    26
980456    27
Name: step, Length: 980457, dtype: int64
<class 'pandas.core.series.Series'> 0         [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...
1         [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...
2         [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...
3         [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...
4         [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...
                                ...                        
980452    [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...
980453    [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...
980454    [[-1, -1, 

rounds                     None
step                       None
field                      None
bombs                      None
coins                      None
agent_1_state              None
agent_1_action             None
agent_2_state              None
agent_2_action             None
agent_3_state              None
agent_3_action             None
agent_0_action             None
agent_0_state              None
explosion_map              None
batch_number               None
agent_0_move_1_step_ago    None
agent_1_move_1_step_ago    None
agent_2_move_1_step_ago    None
agent_3_move_1_step_ago    None
agent_0_move_2_step_ago    None
agent_1_move_2_step_ago    None
agent_2_move_2_step_ago    None
agent_3_move_2_step_ago    None
agent_0_move_3_step_ago    None
agent_1_move_3_step_ago    None
agent_2_move_3_step_ago    None
agent_3_move_3_step_ago    None
agent_0_move_4_step_ago    None
agent_1_move_4_step_ago    None
agent_2_move_4_step_ago    None
agent_3_move_4_step_ago    None
self    

In [40]:
for agent in tqdm(range(4)):
    data["self"] = data[f"agent_{agent}_state"]
    data["others"] = pd.Series(data[[f"agent_{i}_state"
                                         for i in range(0, 4)
                                             if i != agent]]\
                               .values.tolist())
    data[f"agent_{agent}_graph"] = data.apply(MovementGraph, axis=1)

Widget Javascript not detected.  It may not be installed or enabled properly.


KeyboardInterrupt: 

0         [(rule_based_agent_1, 0, True, (15, 1)), (rule...
1         [(rule_based_agent_1, 0, True, (15, 2)), (rule...
2         [(rule_based_agent_1, 0, True, (15, 3)), (rule...
3         [(rule_based_agent_1, 0, False, (15, 3)), (rul...
4         [(rule_based_agent_1, 0, False, (15, 2)), (rul...
                                ...                        
980452    [(random_agent_1, 0, True, (14, 15)), (random_...
980453    [(random_agent_1, 0, True, (14, 15)), (random_...
980454    [(random_agent_1, 0, True, (14, 15)), (random_...
980455    [(random_agent_1, 0, True, (14, 15)), (random_...
980456    [(random_agent_1, 0, True, (14, 15)), (random_...
Length: 980457, dtype: object

In [13]:
def compute_movement_graph_for_step(step):
    for j in range(4):
        step["self"] = step[f"agent_{j}_state"]
        step["others"] = [step[f"agent_{i}_state"] for i in range(1, 4) if i != j]
        step[f"agent_{j}_graph"] = MovementGraph(step)
    return step


fc = FeatureCollector.create_current_version()
def compute_features(step):
    step = compute_movement_graph_for_step(step)
    return fc.compute_feature(step, step)

In [15]:
compute_features(data.iloc[0].copy())

AttributeError: 'Series' object has no attribute 'past_moves'

In [ ]:
res = []
for index, step in data[:5].iterrows():
    data[index] = compute_movement_graph_for_step(step)

data

In [ ]:
MovementGraph(data.iloc[0])

In [ ]:
# added column for agent 4 (instead of agent 0) by mistake
# oops
# drop them

data = data.drop(labels=["agent_4_state", "agent_4_action"], axis=1)

In [ ]:
# set round and step as index
data.index = pd.MultiIndex.from_arrays([data.rounds, data.step])

In [ ]:
for agent_num in tqdm(range(4)):
    agent_name = f"agent_{agent_num}"
    agent_state = f"{agent_name}_state"
    
    # transform the state tuples to columns
    data = data[agent_state].apply(pd.Series).join(data)
    data = data.rename(dict(enumerate(f"{agent_name}_{field_name}" for field_name 
                                          in ["type", "score","bomb_possible", "position"])), axis=1)\
               .drop(labels=[agent_state], axis=1)

In [ ]:
data = pd.read_pickle("./data.pkl")

In [ ]:
data = pd.read_pickle("./data/data.pkl")

In [ ]:
MovementGraph

In [ ]:
data.field.apply(lambda x: pd.Series(x.flatten())).to_pickle("./field.pkl")

In [ ]:
data.explosion_map.apply(lambda x: pd.Series(x.flatten())).to_pickle("./explosion_map.pkl")

In [ ]:
bombs = data.bombs.apply(pd.Series).rename(dict(enumerate([f"bomb_{i}" for i in range(4)])), axis=1)

In [ ]:
bombs.to_pickle("./bombs.pkl")

In [ ]:
bombs = pd.read_pickle("./bombs_expanded.pkl")

In [ ]:
def make_row(bs):
    row = dict()
    for b_num, b in enumerate(bs):
        row[f"bomb_{b_num}_x"] = b[0][0]
        row[f"bomb_{b_num}_y"] = b[0][1]
        row[f"bomb_{b_num}_counter"] = b[1]
    return row

bombs = [make_row(data[["bombs"]].loc[i][0]) for i in tqdm(range(len(data)))]
bombs = pd.DataFrame(bombs)

In [ ]:
data[[c for c in data.columns if c.startswith("agent")]].to_pickle("./agents.pkl")

In [ ]:
agents = pd.read_pickle("./agents.pkl")

In [ ]:
coins = data.coins.apply(pd.Series)

In [ ]:
coins.isna().sum() / len(coins)

In [ ]:
coins = old_coins

In [ ]:
coins = coins[~coins.isna().any(axis=1)]

In [ ]:
cs = pd.DataFrame(index=coins.index)

for i in tqdm(range(coins.shape[1])):
    cs = cs.join(coins[i].apply(pd.Series).rename({0: f"coin_{i}_x", 1:f"coin_{i}_y"}, axis=1))
cs.to_pickle("coins.pkl")

In [ ]:
scores = data[[c for c in data.columns if c.endswith("score")]]
# we dont want the total scores but just the scores recieved in this step
scores = (scores.shift(-1, axis=0) - scores).shift(1, axis=0).fillna(0).astype(int)

In [ ]:
# the problem with this is that the first entry can be negative
# the good thing is that we can just set these values, since there can never be a non-zero score
# in the first step
scores.xs(1, level="step")

In [ ]:
scores[scores.index.get_level_values(1) == 1] = 0
scores.to_pickle("scores.pkl")

In [ ]:
scores = pd.read_pickle("scores.pkl")

In [ ]:
scores.xs(1, level="rounds")

In [ ]:
# max step per round is 400
scores.groupby(level=0).count().max()

In [ ]:
np.roll(np.arange(10), 1, axis=0)

In [ ]:
x = np.array([1, 2, 3, 4, 5, 6])

np.pad(x,(7), mode='constant')[:, :-1]

In [ ]:
def calculate_returns(scores, gamma=.5):
    N = scores.shape[-1]
    exponents = np.stack([np.arange(-i, N - i) for i in np.arange(N)])
    return np.triu(gamma ** exponents).dot(scores.T).T 

In [ ]:
def calculate_returns_for_all_agents(scores, agents, gammas):
    if not isinstance(gammas, list):
        gammas = [gammas]
    for gamma in tqdm(gammas):
        for agent in tqdm(agents):
            col_name = f"{agent}_return_gamma_{str(gamma).replace('.', '_')}"

            scores[col_name] = scores\
                                  .agent_0_score\
                                  .groupby(level=0)\
                                  .apply(lambda v: pd.Series(calculate_returns(v), 
                                            index=np.arange(len(v)) + 1))

In [ ]:
calculate_returns_for_all_agents(scores, agents=[f"agent_{i}" for i in range(4)], gammas=[0.6, 0.8, 0.95])

In [ ]:
scores.to_pickle("returns.p")

In [ ]:
bar = pd.Series(calculate_returns(scores.agent_0_score.xs(1)))
bar.index += 1

In [ ]:
bar

In [ ]:
scores.agent_0_return_gamma_0_5.xs(1)

In [ ]:
scores.agent_0_score.shape

In [ ]:
calculate_returns(scores.agent_0_score.unstack(level=-1).fillna(0), gamma=0.5).reshape((980457)).shape

In [ ]:
pd.DataFrame(calculate_returns(scores.agent_0_score.unstack(level=-1).fillna(0), gamma=0.5), index=scores.agent_0_score.index)

In [ ]:
scores.agent_0_score.unstack(level=-1).fillna(0).shape

In [ ]:
np.array(2) ** np.arange(4)